In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup
import re

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
fhand = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(fhand, 'html.parser')

In [28]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df = df[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [6]:
df.shape

(289, 3)

In [11]:
df = df[~df[1].isin(['Not assigned'])]

In [12]:
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [13]:
df.shape

(212, 3)

In [14]:
df.reset_index(drop=True, inplace=True)


In [15]:
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park


In [16]:
def concatenate_neighbourhood(x):
    conc = ""
    for i in range(len(x)-1):
        conc = conc + x.iloc[i] + ", "
    conc += x.iloc[-1]
    return conc

def fix_borough(x):
    ref = x.iloc[0]
    for i in range(1, len(x)):
        if ref != x.iloc[i]:
            for i in x:
                print(x)
    return ref

In [20]:
df = df.groupby([0]).agg({1: lambda x: fix_borough(x), 
                                   2: lambda x: concatenate_neighbourhood(x)})
df.head()

,1,2
0,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [21]:
rows = df.shape[0]
print('There are %d rows in the dataframe' % rows)


There are 104 rows in the dataframe


In [29]:
df.to_csv('ca-neighborhoods.csv')